# 3rd Level Model Structure: Simulator

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output

from copy import deepcopy

In [2]:
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import trans_product_mole, trans_product_mass, beautify
from utility.model_utility import add_dual, update_dual, check_DOF
from utility.data_utility import cal_cnumber

model = pe.ConcreteModel()

# Global Set

In [3]:
model.TRAY = pe.RangeSet(1,8)

# Construct Reactive Stages

In [4]:
model.reactive = pe.Block(model.TRAY,rule=reactive_stage_rule)

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[1].x_
| reactive[1].y_
| reactive[1].x
| reactive[1].y
| reactive[1].z
| reactive[1].L
| reactive[1].V
| reactive[1].F
| reactive[1].H_L_
| reactive[1].H_V_
| reactive[1].H_L
| reactive[1].H_V
| reactive[1].H_F
| reactive[1].T
| reactive[1].T_F
| reactive[1].P
| reactive[1].f_V
| reactive[1].f_L
| reactive[1].cat
| reactive[1].Q_main
| reactive[1].r_total_comp
| reactive[1].PR_L
| reactive[1].PR_V
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[1].kinetics_block.k_FT
| reactive[1].kinetics_block.r_FT_total
| reactive[1].kinetics_block.g0_FT
| reactive[1].kinetics_block.alpha
| reactive[1].kinetics_block.r_FT_cnum
| reactive[1].kinetics_block.r_FT_comp
| reactive[1].kinetics_block.k_WGS
| reactive[1].kinetics_block.Ke_WGS
| reactive[1].kinetics_b

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[4].MPCC.s_L
| reactive[4].MPCC.s_V
| reactive[4].MPCC.pf
| reactive[4].MPCC.rho
--------------------------------------------------
> Spliting pressure used in VLE

> Importing Reactive Stage......
> Adding the following local variable:
------------------------------------
| reactive[5].x_
| reactive[5].y_
| reactive[5].x
| reactive[5].y
| reactive[5].z
| reactive[5].L
| reactive[5].V
| reactive[5].F
| reactive[5].H_L_
| reactive[5].H_V_
| reactive[5].H_L
| reactive[5].H_V
| reactive[5].H_F
| reactive[5].T
| reactive[5].T_F
| reactive[5].P
| reactive[5].f_V
| reactive[5].f_L
| reactive[5].cat
| reactive[5].Q_main
| reactive[5].r_total_comp
| reactive[5].PR_L
| reactive[5].PR_V
------------------------------------

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| reactive[5].kin

# Construct a single condenser

In [5]:
model.condenser = pe.Block(rule=condenser_stage_rule)

| Importing Condenser Stage......
| Adding the following local variable:
------------------------------------
| condenser.T
| condenser.T_F
| condenser.P
| condenser.Q_main
| condenser.x_
| condenser.y_
| condenser.x
| condenser.y
| condenser.z
| condenser.L
| condenser.W
| condenser.V
| condenser.F
| condenser.H_L_
| condenser.H_V_
| condenser.H_L
| condenser.H_V
| condenser.H_F
| condenser.f_V
| condenser.f_L
| condenser.PR_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.energy_block.dH_F
| condenser.energy_block.dH_V
| condenser.energy_block.dH_L
| condenser.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| condenser.VLE_block.n_ave
| condenser.VLE_block.n_ave_cal
| condenser.VLE_block.Hen
| condenser.VLE_block.Hen0
| 

# Construct a single reboiler

In [6]:
model.reboiler = pe.Block(rule=reboiler_stage_rule)

> Importing Non Reactive Stage......
> Adding the following local variable:
------------------------------------
| reboiler.T_F
| reboiler.P
| reboiler.Q_main
| reboiler.x_
| reboiler.y_
| reboiler.x
| reboiler.y
| reboiler.z
| reboiler.L
| reboiler.V
| reboiler.F
| reboiler.H_L_
| reboiler.H_V_
| reboiler.H_L
| reboiler.H_V
| reboiler.T
| reboiler.H_F
| reboiler.f_V
| reboiler.f_L
------------------------------------

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.energy_block.dH_F
| reboiler.energy_block.dH_V
| reboiler.energy_block.dH_L
| reboiler.energy_block.dH_vap
--------------------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| reboiler.VLE_block.P_VLE
| reboiler.VLE_block.n_ave
| reboiler.VLE_block.n_ave_cal
| reboiler.VLE_block.Hen
| reboiler.VLE_block.Hen0
| reboiler.VLE_block.gamma
| r

# Linking Stage Variables

### Vapor Between Reactive Stages

In [7]:
def V_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].V['in'] == model.reactive[j+1].V['out']
model.V_between_con = pe.Constraint(model.TRAY,rule=V_between_rule)

def Vy_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].y_['in',i] == model.reactive[j+1].y[i]
model.Vy_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Vy_between_rule)

def Vh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j].H_V_['in'] == model.reactive[j+1].H_V
model.Vh_between_con = pe.Constraint(model.TRAY,rule=Vh_between_rule)

### Liquid Between Reactive Stages

In [8]:
def L_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].L['in'] == model.reactive[j].L['out']
model.L_between_con = pe.Constraint(model.TRAY,rule=L_between_rule)

def Lx_between_rule(model,j,i):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].x_['in',i] == model.reactive[j].x[i]
model.Ly_between_con = pe.Constraint(model.TRAY,m.COMP_TOTAL,rule=Lx_between_rule)

def Lh_between_rule(model,j):
    if j == model.TRAY.last(): return pe.Constraint.Skip
    return model.reactive[j+1].H_L_['in'] == model.reactive[j].H_L
model.Lh_between_con = pe.Constraint(model.TRAY,rule=Lh_between_rule)

### Condenser Vapor

In [9]:
def V_condenser_rule(model):
    return model.reactive[model.TRAY.first()].V['out'] == model.condenser.V['in']
model.V_condenser_con = pe.Constraint(rule=V_condenser_rule)

def Vy_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].y[i] == model.condenser.y_['in',i]
model.Vy_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_condenser_rule)

def Vh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_V == model.condenser.H_V_['in']
model.Vh_condenser_con = pe.Constraint(rule=Vh_condenser_rule)

### Condenser Liquid

In [10]:
def L_condenser_rule(model):
    return model.reactive[model.TRAY.first()].L['in'] == model.condenser.L['out']
model.L_condenser_con = pe.Constraint(rule=L_condenser_rule)

def Lx_condenser_rule(model,i):
    return model.reactive[model.TRAY.first()].x_['in',i] == model.condenser.x[i]
model.Lx_condenser_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_condenser_rule)

def Lh_condenser_rule(model):
    return model.reactive[model.TRAY.first()].H_L_['in'] == model.condenser.H_L
model.Lh_condenser_con = pe.Constraint(rule=Lh_condenser_rule)

### Reboiler Vapor

In [11]:
def V_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].V['in'] == model.reboiler.V['out']
model.V_reboiler_con = pe.Constraint(rule=V_reboiler_rule)

def Vy_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].y_['in',i] == model.reboiler.y[i]
model.Vy_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Vy_reboiler_rule)

def Vh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_V_['in'] == model.reboiler.H_V
model.Vh_reboiler_con = pe.Constraint(rule=Vh_reboiler_rule)

### Reboiler Liquid

In [12]:
def L_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].L['out'] + 1e-6 == model.reboiler.L['in']
model.L_reboiler_con = pe.Constraint(rule=L_reboiler_rule)

def Lx_reboiler_rule(model,i):
    return model.reactive[model.TRAY.last()].x[i] == model.reboiler.x_['in',i]
model.Lx_reboiler_con = pe.Constraint(m.COMP_TOTAL,rule=Lx_reboiler_rule)

def Lh_reboiler_rule(model):
    return model.reactive[model.TRAY.last()].H_L == model.reboiler.H_L_['in']
model.Lh_reboiler_con = pe.Constraint(rule=Lh_reboiler_rule)

In [13]:
model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY)\
                         - model.reboiler.MPCC.pf ,sense=pe.maximize)
# model.obj = pe.Objective(expr = sum(model.reactive[j].T - model.reactive[j].MPCC.pf for j in model.TRAY) ,sense=pe.maximize)
# model.obj = pe.Objective(expr = sum(model.reactive[j].T for j in model.TRAY) ,sense=pe.maximize)

In [14]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


# Load from Reactive Flash solutions

In [15]:
with open('../../saved_solutions/reactive_flash_MPCC_P_pf_200C.pickle', 'rb') as f:
    results_imported = pickle.load(f)
results_changed = deepcopy(results_imported)

### Duplicate variable solution and bounds multiplier

In [16]:
results_changed.Solution.Variable = {}
for i in list(results_imported.Solution.Variable.keys()):
    for j in model.TRAY:
        results_changed.Solution.Variable['reactive[{}].{}'.format(j,i)] = \
        results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [17]:
results_changed.Solution.Constraint = {}
for i in list(results_imported.Solution.Constraint.keys()):
    for j in model.TRAY:
        results_changed.Solution.Constraint['reactive[{}].{}'.format(j,i)] = \
        results_imported.Solution.Constraint[i]

# Load from Condenser solutions

In [18]:
with open('../../saved_solutions/condenser_MPCC_P_pf_30C.pickle', 'rb') as f:
    condenser_results_imported = pickle.load(f)

### Duplicate variable solution and bounds multiplier

In [19]:
for i in list(condenser_results_imported.Solution.Variable.keys()):
    results_changed.Solution.Variable['condenser.{}'.format(i)] = \
    condenser_results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [20]:
for i in list(condenser_results_imported.Solution.Constraint.keys()):
    results_changed.Solution.Constraint['condenser.{}'.format(i)] = \
    condenser_results_imported.Solution.Constraint[i]

# Load from Reboiler solutions

In [21]:
with open('../../saved_solutions/reboiler_MPCC_P_pf_200C.pickle', 'rb') as f:
    reboiler_results_imported = pickle.load(f)

### Duplicate variable solution and bounds multiplier

In [22]:
for i in list(reboiler_results_imported.Solution.Variable.keys()):
    results_changed.Solution.Variable['reboiler.{}'.format(i)] = \
    reboiler_results_imported.Solution.Variable[i]

### Duplicate constraint multiplier

In [23]:
for i in list(reboiler_results_imported.Solution.Constraint.keys()):
    results_changed.Solution.Constraint['reboiler.{}'.format(i)] = \
    reboiler_results_imported.Solution.Constraint[i]

In [24]:
model.solutions.load_from(results_changed)

RuntimeError: Symbol reboiler.magic is missing from model unknown

# Fixing Redundent Stream Variables

In [ ]:
# condenser
model.condenser.VLE_block.n_ave.fix(4)

model.condenser.F.fix(0)
model.condenser.T_F.fix(300+273.15)
model.condenser.z.fix(0)

model.condenser.V['P'].fix(0)
model.condenser.L['in'].fix(0)
for i in m.COMP_TOTAL: model.condenser.x_['in',i].fix(0)
model.condenser.H_L_['in'].fix(0)

In [ ]:
# reboiler
model.reboiler.VLE_block.n_ave.fix(20)

model.reboiler.F.fix(0)
model.reboiler.T_F.fix(300+273.15)
model.reboiler.z.fix(0)

model.reboiler.V['P'].fix(0)
model.reboiler.V['in'].fix(0)
for i in m.COMP_TOTAL: model.reboiler.y_['in',i].fix(0)
model.reboiler.H_V_['in'].fix(0)
model.reboiler.L['P'].fix(0)

# Load Operating Parameters

In [ ]:
# condenser
model.condenser.P.fix(19)
# model.condenser.T.fix(30+273.15)
model.condenser.T.fix(30+273.15)
model.condenser.L['out'].fix(0)

# reboiler
model.reboiler.P.fix(20)
# model.reboiler.T.fix(30+273.15)
model.reboiler.T.fix(200+273.15)
# model.reboiler.Q_main.fix(0)

# reactive stage
for j in model.reactive:
    model.reactive[j].cat.fix(3000)
    model.reactive[j].P.fix(20)
    model.reactive[j].VLE_block.n_ave.fix(20)
    
    model.reactive[j].F.fix(1)
    model.reactive[j].T_F.fix(200+273.15)
    model.reactive[j].z['CO'].fix(1/(1+2)-0/2)
    model.reactive[j].z['H2'].fix(2/(1+2)-0/2)
    model.reactive[j].z['C30H62'].fix(0)
    model.reactive[j].V['P'].fix(0)
    model.reactive[j].L['P'].fix(0)
    # model.reactive[j].Q_main.fix(0)
    model.reactive[j].T.setub(200+273.15)

In [ ]:
check_DOF(pe,model)

In [ ]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma86'

opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'

opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

# opt.options['recalc_y'] = 'yes'
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

In [ ]:
beautify(pe,model)

# Construct the simulator

In [ ]:
style = {'description_width': 'initial'}
layout1 = widgets.Layout(width='200px')


t_control = {}
for i in model.reactive:
    t_control[i] = widgets.FloatSlider(min=100+273.15,max=250+273.15,value=200+273.15,step=5,description='T {:}'.format(i),style=style)
L_P_control = {}
for i in model.reactive:
    L_P_control[i] = widgets.FloatSlider(min=0,max=0.1,step=0.01,description='Side Draw {:}'.format(i),style=style) 
cat_control = {}
for i in model.reactive:
    cat_control[i] = widgets.FloatSlider(min=0,max=5000,value=3000,step=100,description='Cat {:}'.format(i),style=style)   
feed_control = {}
for i in model.reactive:
    feed_control[i] = widgets.FloatSlider(min=0,max=2,value=1,step=0.1,description='Feed {:}'.format(i),style=style)   


reflux_control = widgets.FloatSlider(min=0,max=5,step=0.1,description='Reflux',style=style)  
reboiler_temperature_control = widgets.FloatSlider(min=200+273.15,max=350+273.15,step=0.1,description='Temperature'.format(i),style=style)
bottom_control = widgets.FloatSlider(min=0,max=0.1,step=0.01,description='Bottom Rate'.format(i),style=style)
rho_control1 = widgets.BoundedFloatText(min=1,max=10000,step=10,description='Penalty Weight Reactive'.format(i),style=style)
rho_control2 = widgets.BoundedFloatText(min=1,max=10000,step=10,description='Penalty Weight Reboiler'.format(i),style=style)

solver_control = widgets.Checkbox(
    value=True,
    description='Display IPOPT output',
    disabled=False
)

run_solve = widgets.Button(
    description='Run Solve',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run Solve',
    icon='check'
)

plot_button = widgets.Button(
    description='Plot Distribution',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Plot Distribution',
    icon='check'
)

clear_plot_button = widgets.Button(
    description='Clear Plots',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Plot Distribution',
    icon='legal'
)

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

In [ ]:
out3.layout.height = '1000px'

In [ ]:
def button_click(event):
    out1.clear_output()
    
    model.condenser.L['out'].fix(reflux_control.value)
    
    for i in model.reactive:
        model.reactive[i].T.setub(t_control[i].value)
        model.reactive[i].L['P'].fix(L_P_control[i].value) 
        model.reactive[i].MPCC.rho = rho_control1.value
        model.reactive[i].cat.fix(cat_control[i].value)
        model.reactive[i].F.fix(feed_control[i].value)
    
    model.reboiler.T.fix(reboiler_temperature_control.value)
    model.reboiler.MPCC.rho = rho_control2.value
    model.reactive[model.TRAY.last()].L['P'].fix(bottom_control.value)

    with out1:
        results = opt.solve(model,tee=solver_control.value)
    update_dual(pe,model)
    with out2:
        beautify(pe,model)
        
run_solve.on_click(button_click)

def plot_click(event):
    with out3:
        plot_distribution(model)

plot_button.on_click(plot_click)

def clear_plot_click(event):
    out2.clear_output()
    out3.clear_output()
    with out2:
        beautify(pe,model)
    with out3:
        plot_distribution(model)

clear_plot_button.on_click(clear_plot_click) 

In [ ]:
cnumber_range = range(1,57)
def trans_cnumber(dic):
    molefraction = {}
    for i in cnumber_range:
        molefraction[i] = []
    for i in m.COMP_ORG:
        molefraction[cal_cnumber(i)].append(np.array(dic[i]))
    for i in cnumber_range:
        molefraction[i] = np.sum(molefraction[i],0)
    tmp = []
    for i in cnumber_range:
        tmp.append(molefraction[i])
    return tmp 
    
def plot_distribution(model):
    
    g_data = []; d_data = []; l_out_data = {}; l_P_data = {}; b_data = []
    cd_x_data = []; rf_x_data = {}; rb_x_data = []

    g_data = trans_cnumber({i:model.condenser.V['out'].value*model.condenser.y[i].value for i in m.COMP_TOTAL})
    d_data = trans_cnumber({i:model.condenser.L['P'].value*model.condenser.x[i].value for i in m.COMP_TOTAL})
    for j in model.reactive:
        l_out_data[j] = trans_cnumber({i:model.reactive[j].L['out'].value*model.reactive[j].x[i].value for i in m.COMP_TOTAL})
        l_P_data[j] = trans_cnumber({i:model.reactive[j].L['P'].value*model.reactive[j].x[i].value for i in m.COMP_TOTAL})
    b_data = trans_cnumber({i:model.reboiler.L['out'].value*model.reboiler.x[i].value for i in m.COMP_TOTAL})

    cd_x_data = trans_cnumber({i:model.condenser.x[i].value for i in m.COMP_TOTAL})
    for j in model.reactive:
        rf_x_data[j] = trans_cnumber({i:model.reactive[j].x[i].value for i in m.COMP_TOTAL})
    rb_x_data = trans_cnumber({i:model.reboiler.x[i].value for i in m.COMP_TOTAL})
                
    fig, (ax,ax2) = plt.subplots(1,2,figsize=(16,7))
    ax.plot(cnumber_range,g_data,'co-')
    ax.plot(cnumber_range,d_data,'go-')
    for j in model.reactive:
        ax.plot(cnumber_range,l_P_data[j],'bo-',alpha = (j+1)/(len(model.TRAY)+1),marker = r'${:}$'.format(j),markersize=10)
    ax.plot(cnumber_range,b_data,'ro-')
    
    ax.set_xlim(0, 40)
    ax.set_yscale("log")
    ax.set_ylim(1e-12, 1)
    # ax.legend(['Vapor','Distillate',*['P{:}'.format(i) for i in model.reactive],'Bottom'],fontsize=14,loc=1)
    ax.set_title('Product Distribution',fontsize=18)

    ax.set_ylabel('Molar Flow (kmol/s)', color='K',fontsize=18)

    ax2.plot(cnumber_range,cd_x_data,'go-')
    for j in model.reactive:
        ax2.plot(cnumber_range,rf_x_data[j],'bo-',alpha = (j+1)/(len(model.TRAY)+1),marker = r'${:}$'.format(j),markersize=10)
    ax2.plot(cnumber_range,rb_x_data,'ro-')  

    ax2.set_xlim(0, 40)
    ax2.set_ylim(0, 0.5)
    ax2.legend(['Condenser',*['Stage{:}'.format(i) for i in model.reactive],'Reboiler'],fontsize=14,loc=1)
    ax2.set_title('Liquid Composition (Mole)',fontsize=18)

    ax2.set_ylabel('Molar Fraction', color='K',fontsize=18)
    ax2.set_xlabel('Carbon Number', color='K',fontsize=18)

    ax.grid()
    ax2.grid()
    plt.show()

In [ ]:
t_control_vbox = widgets.VBox([t_control[i] for i in model.reactive] + [cat_control[i] for i in model.reactive])
L_P_control_vbox = widgets.VBox([L_P_control[i] for i in model.reactive] + [feed_control[i] for i in model.reactive])

linebreak = widgets.Label('-'*120)

reactive_interface = widgets.HBox([t_control_vbox,L_P_control_vbox])
reboiler_interface = widgets.HBox([reboiler_temperature_control,bottom_control])
solver_interface = widgets.HBox([run_solve,plot_button,clear_plot_button,solver_control])
rho_interface = widgets.HBox([rho_control1,rho_control2])

interface = widgets.VBox([widgets.Label('Condenser'),linebreak,reflux_control,linebreak,\
                          widgets.Label('Reactive'),linebreak,reactive_interface,linebreak,\
                          widgets.Label('Reboiler'),linebreak,reboiler_interface,linebreak,\
                          rho_interface,linebreak,\
                          solver_interface])

In [ ]:
display(interface,out2,out3)

In [ ]:
display(out1)